# Lab 1: Ingesta de datos con Logstash

En este ejercicio vamos a poner en práctica lo aprendido hasta haora:

* Creación de index templates.
* Ingesta de datos con Logstash.
* Transformación y enriquecimiento de datos con Logstash.

Para ello vamos a ingestar un conjunto de datos sobre vuelos de aviones comerciales, de esta forma prepararemos los datos que utilizaremos en próximos notebooks.

Antes de empezar vamos a familiarizarnos con los datos que quermos ingestar. Para ello vamos a ver el contenido del fichero que contiene el conjunto de datos fuente, trips.csv que puedes encontrar aquí: http://127.0.0.1:8889/edit/work/data/elasticsearch/trips/trips.csv

Para explorar los datos, vamos a utilizar la librería pandas de python. Vemos que campos contiene y de que tipo son estos campos:

In [4]:
import pandas as pd

trips = pd.read_csv("../../data/elasticsearch/trips/trips.csv")

print(trips.dtypes)

StartAirport        object
EndAirport          object
TripID               int64
Type                object
ActivityID           int64
ActivityCostAUD    float64
AirlineCode         object
Aircraft            object
ServiceClass        object
FlightNumber         int64
StartCountry        object
StartCityName       object
StartLat           float64
StartLong          float64
StartDate           object
StartTime           object
EndCountry          object
EndCityName         object
EndLat             float64
EndLong            float64
EndDate             object
EndTime             object
Stops               object
DistanceKM           int64
dtype: object


Vamos a mostrar una muestra de los cinco primeros registros para hacernos una idea de su contenido:

In [5]:
print(trips.head(5)) 

  StartAirport EndAirport     TripID Type  ActivityID  ActivityCostAUD  \
0          CBR        MEL  306007947  Air  1141935494          1241.36   
1          MEL        CBR  306007947  Air  1141935494          1241.36   
2          SYD        MEL  305316367  Air  1140039658           502.00   
3          CBR        SYD  305312206  Air  1140385947          1313.16   
4          MEL        SYD  305312206  Air  1140269701           350.00   

  AirlineCode                                         Aircraft ServiceClass  \
0          QF         Boeing 737-800 (winglets) Passenger/BBJ2      Economy   
1          QF                                   Boeing 717-200      Economy   
2          VA         Boeing 737-800 (winglets) Passenger/BBJ2      Economy   
3          QF  De Havilland (Bombardier) DHC-8-300 Dash 8 / 8Q      Economy   
4          VA         Boeing 737-800 (winglets) Passenger/BBJ2      Economy   

   FlightNumber     ...     StartDate StartTime  EndCountry  EndCityName  \
0   

## Ejercicio 1: Ceración del index template para modelar los datos 

Ahora que ya tenemos una idea de la información que queremos indexar, vamos a crear el modelo de datos que va a alojar la información según nuestras necesidades de explotación.

Para ello crea un index template con nombre trips que cumpla las siguientes condiciones:

* El template se aplicará para todos lo índices cuyo nombre se llame "trips".
* Todos los campos de tipo string se almacenarán como tipo keyword. Sugerencia, utiliza una dynamic template para definir esta condición.
* Los campos EndAirportGeo y StartAirportGeo que vamos a crear los vamos a almacenar con tipo geo_point para poder explotarlos correctamente utilizando consultas geo espaciales. En este enlace puedes ver como definir este tipo de datos: https://www.elastic.co/guide/en/elasticsearch/reference/current/geo-point.html
* Los campos DistanceKM y ActivityCostAUD los almacenaremos como tipo integer para poder hacer agregaciones sobre ellos.
* Los campos StartTime, EndTime serán de tipo date con formato horaio. Consultar los datos para comprobar que formatos hay que indicar al crear el campo. En este enlace puedes ver como definir el campo de tipo date: https://www.elastic.co/guide/en/elasticsearch/reference/current/date.html y en este otro como espedificar el formato de las fechas: https://www.elastic.co/guide/en/elasticsearch/reference/current/date.html#multiple-date-formats
* Los campos StartDate y EndDate serán de tipo date con formato de fecha dia/mes/año. Consultar los datos para comprobar que formatos hay que indicar al crear el campo.

Una vez creado, consulta en Kibana que todo se ha creado correctamente.

In [6]:
!curl -X PUT http://elasticsearch:9200/_index_template/trips -H 'Content-Type: application/json' -d ' \
{ \
  "index_patterns": ["trips"], \
  "template": { \
    "mappings": { \
      "dynamic_templates": [ \
        { \
          "strings_as_keywords": { \
            "match_mapping_type": "string", \
            "mapping": { "type": "keyword" } \
          } \
        } \
      ], \
      "properties": { \
        "EndAirportGeo": { "type": "geo_point" }, \
        "StartAirportGeo": { "type": "geo_point" }, \
        "DistanceKM": { "type": "integer" }, \
        "ActivityCostAUD": { "type": "integer" }, \
        "StartTime": { \
          "type":   "date", \
          "format": "HH:mm:ss||H:mm:ss||HH:mm||H:mm" \
        }, \
        "EndTime": { \
          "type":   "date", \
          "format": "HH:mm:ss||H:mm:ss||HH:mm||H:mm" \
        }, \
        "StartDate": { \
          "type":   "date", \
          "format": "dd/MM/yy||dd/M/yy||d/MM/yy||d/M/yy" \
        }, \
        "EndDate": { \
          "type":   "date", \
          "format": "dd/MM/yy||dd/m/yy||d/MM/yy||d/M/yy" \
        } \
      } \
    } \
  } \
}'      

{"acknowledged":true}

## Ejercicio 2: Configurar la ingesta de datos en Logstahs

Antes de levantar el servicio de Logstash es enecesario configurarlo. Para ello encontrarás dentro de la carpeta work/data/elasticsearch/trips/pipeline/ el fichero de configuración logstash-trips.conf editalo para modificar la configuración.

* Primero añade el imput plugin, que será de tipo fichero y parseará los ficheros que encuentre en /tmp/data/. En este enlace tienes toda la documentación de como configurar este plugin: https://www.elastic.co/guide/en/logstash/current/plugins-inputs-file.html
* Añade los filtros para realizar las siguientes operaciones con los datos:
    * Indica que el origen es un fichero csv utilizando el csv plugin. En este enlace puedes encontrar como configurar este plugin: https://www.elastic.co/guide/en/logstash/current/plugins-filters-csv.html Indica que ignore el header y especifica cuales seran los nombres de las columnas: StartAirport","EndAirport","TripID","Type","ActivityID","ActivityCostAUD","AirlineCode","Aircraft","ServiceClass","FlightNumber","StartCountry","StartCityName","StartLat","StartLong","StartDate","StartTime","EndCountry","EndCityName","EndLat","EndLong","EndDate","EndTime","Stops","DistanceKM".
    * Modifica los mensajes utilizando el plugin mutate (https://www.elastic.co/guide/en/logstash/current/plugins-filters-mutate.html) para añadir dos campos que vamos a calcular de la siguiente forma:
        * StartAirportGeo será el resultado de concatenar StartLat y StartLong separados por una coma.
        * EndAirportGeo será el resultado de concatenar EndLat y EndLong separados por una coma.
    * Por último, con el mismo plugin mutate, elimina los siguientes campos del mensaje: "host", "@version", "@timestamp", "message", "StartLat", "EndLat", "StartLong", "EndLong".
* Indica que los mensajes procesados se almacenarán en elasticsearch en el índice trips utilizando el plugin output de Elasticsearch: https://www.elastic.co/guide/en/logstash/current/plugins-outputs-elasticsearch.html.



Configuración de logstash:

`
input {
    file {
        path => "/tmp/data/*"
    }
}
`

`
filter {
    csv {
        source => "message"
        columns => ["StartAirport","EndAirport","TripID","Type","ActivityID","ActivityCostAUD","AirlineCode","Aircraft","ServiceClass","FlightNumber","StartCountry","StartCityName","StartLat","StartLong","StartDate","StartTime","EndCountry","EndCityName","EndLat","EndLong","EndDate","EndTime","Stops","DistanceKM"]
        skip_header => true
    }

    mutate {
        add_field => {
            "StartAirportGeo" => "%{StartLat},%{StartLong}"
            "EndAirportGeo" => "%{EndLat},%{EndLong}"
            }
        remove_field => ["host", "@version", "@timestamp", "message", "StartLat", "EndLat", "StartLong", "EndLong"]
    }
}
`

`
output {
    elasticsearch {
    hosts => "elasticsearch:9200"
    index => "trips"
  } 
}
`

## Ejercicio3: Levantar la imagen de docker de Logstash

Una vez configurado, levanta la imagen de docker de logstash montando los volúmenes que contienen la configuración y la carpeta donde dejaremos los datos:

In [4]:
docker run --rm -it --network=curso-els-paradigma_default \
    -v /Users/rgarrote/desarrollo/cursoELS/curso-els-paradigma/work/data/elasticsearch/trips/pipeline/:/usr/share/logstash/pipeline/ \
    -v /Users/rgarrote/desarrollo/cursoELS/curso-els-paradigma/work/data/elasticsearch/trips/data/:/tmp/data/ \
docker.elastic.co/logstash/logstash:8.3.3

/bin/sh: 1: docker: not found


Copia el fichero de trips que vimos al principo del notebook en la carpeta work/data/elasticsearch/trips/data/. Una vez que copies el fichero Logstash lo procesará e insertará los datos en el índice de Elasticsearch indicado en la configuración.

* Entra en Kibana para comprobar que se ha creado correctamente el índice. Comprueba el mapping type.
* ¿Cuántos documentos a insertado?
* Realiza una consulta para mostrar 10 elementos insertados en el índice. Comprueba que el contenido de los campos creados son correctos.